# Advanced Chunking Exercise

You will implement a RAG application for long and messy legal documents. You will implement the best practices you learned so far, including semantic chunking, and chunk enrichment. Then, you will implement semantic search and response generation with citation to the original documents.

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich) to make the output more readable, and supress warning messages.

In [1]:
from rich.console import Console
from rich_theme_manager import Theme, ThemeManager
import pathlib

theme_dir = pathlib.Path("../themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

In [2]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Loading complex PDF documents

You will load a complex legal PDF document from the [case.law](https://case.law/) website. This website has millions of legal documents, and we will load a random PDF file from that site with more than 1,000 pages. 

To parse the PDF file you will use a PDF processor library, [pymupdf4llm](https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/), which makes it easy to extract text and other media from PDF files for RAG applications. 

In [3]:
import pymupdf4llm

import requests
import os

random_doc_number = 196
url = f"https://static.case.law/wash-app/{random_doc_number}.pdf"
response = requests.get(url)

data_folder = "data"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

with open(os.path.join(data_folder, f"{{random_doc_number}}.pdf"), "wb") as file:
    file.write(response.content)

md_text = pymupdf4llm.to_markdown(f"data/{{random_doc_number}}.pdf", page_chunks=True)

Processing data/{random_doc_number}.pdf...
[                                        ] (0/1118[                                        ] (   1/111[                                        ] (   2/111[                                        ] (   3/111[                                        ] (   4/111[                                        ] (   5/111[                                        ] (   6/111[                                        ] (   7/111[                                        ] (   8/111[                                        ] (   9/111[                                        ] (  10/111[                                        ] (  11/111[                                        ] (  12/111[                                        ] (  13/111[                                        ] (  14/111[                                        ] (  15/111[                                        ] (  16/111[                                        ] (  17/111[                      

### Show a ramdom page from the document

Let's check a random page from the PDF document and print its image and the extracted text.

In [4]:
import fitz
from IPython.display import display, HTML

random_page_number = 149
## Convert the PDF to an PNG image
pdf_path = "data/196.pdf"
pdf_document = fitz.open(pdf_path)
page = pdf_document.load_page(random_page_number)  # Page numbering starts from 0
pix = page.get_pixmap()
pix.save("random_page.png")
pdf_document.close()

# Text content
text_content = f"""
<h3>Extracted Text</h3>
<p>{md_text[random_page_number]["text"]}</p>
"""

# HTML layout for two columns to show the image and text side by side
html_content = f"""
<div style="display: flex; align-items: center;">
    <div style="flex: 60%; padding: 5px;">
        <img src="{'random_page.png'}" style="max-width: 100%; height: auto;"/>
    </div>
    <div style="flex: 40%; padding: 5px;">
        {text_content}
    </div>
</div>
"""

# Display in Jupyter notebook
display(HTML(html_content))

You can see that the PDF processor extracts additional information on the document such as title, page count, etc. We can use this metadata for the metadata of our chunks in the vector database.

In [5]:
console.print(md_text[random_page_number])

{
    'metadata': {
        'format': 'PDF 1.3',
        'title': 'Washington Appellate Reports volume 196',
        'author': 'anonymous',
        'subject': 'unspecified',
        'keywords': '',
        'creator': 'Harvard Library Innovation Lab',
        'producer': 'ReportLab PDF Library - www.reportlab.com',
        'creationDate': "D:20190924090929+00'00'",
        'modDate': "D:20190924090929+00'00'",
        'trapped': '',
        'encryption': None,
        'file_path': 'data/{random_doc_number}.pdf',
        'page_count': 1118,
        'page': 150
    },
    'toc_items': [],
    'tables': [],
    'images': [
        {
            'number': 0,
            'bbox': Rect(0.0, 50.0, 396.0, 561.760009765625),
            'transform': (396.0, 0.0, -0.0, 611.760009765625, 0.0, 0.0),
            'width': 1650,
            'height': 2549,
            'colorspace': 1,
            'cs-name': 'DeviceGray',
            'xres': 96,
            'yres': 96,
            'bpc': 1,
            'size': 37219
        }
    ],
    'graphics': [],
    'text': 'trial court that the jury authorized only this latter crime as\n\na basis for sentencing.21\n\n      -
4 The State argues that Morales’s reliance on Wil-\n\nliams-Walker is misplaced because CrR 7.8 was not 
dis-\n\ncussed in that case. That is the court rule on which the trial\n\ncourt relied to correct the jury verdict.
This argument is not\n\nconvincing.\n\n       - 5 First, the fact that the supreme court did not discuss\n\nCrR 7.8
in Williams-Walker is not a persuasive distinguish-\n\ning factor. There, the court held that the jury trial 
right\n\nincluded the right to be sentenced only on a basis autho-\n\nrized by a jury’s verdict. It did so on facts
that are not\n\nmaterially distinguishable from those here.\n\n      - 6 There, the jury verdict forms stated the 
“deadly\n\nweapon” enhancement, not the more serious “firearm” en-\n\nhancement. Trial courts in some of the cases 
sentenced on\n\nthe basis of the latter, not the former, enhancement. The\n\ncourt held that was error.\n\n     - 7
Here, the jury verdict stated Child Molestation in\n\nthe Second Degree, not the more serious Child 
Molestation\n\nin the First Degree. The trial court sentenced on the basis of\n\nthe more serious crime, not the 
one in the jury verdict.\n\n     - 8 The underlying principle is the same: the jury ver-\n\ndict authorized only a 
sentence based on that verdict. The\n\ncourt based the sentence on a crime not authorized by the\n\njury 
verdict.\n\n     - 9 Second, we deal later in this opinion with the ques-\n\ntion whether CrR 7.8 is a proper 
remedy to correct an\n\narguably erroneous jury verdict. That discussion deals more\n\nfully with the State’s 
argument.\n\n     - 0 Given that there was an arguably erroneous jury\n\nverdict, we must decide whether the court 
had the authority\n\nto change it. Two supreme court cases provide guidance.\n\n\n-----\n\n',
    'words': []
}

## Split the documents into Chunks

You will use the statistical chunker that we used in the hands-on lab. However, we want an encoder that is trained on legal document and can generate better embedding vectors to improve the retrieval results. For this exercise you will an encoder from Hugging Face hub: https://huggingface.co/nlpaueb/legal-bert-base-uncased.

In [6]:
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder(
    name="nlpaueb/legal-bert-base-uncased"
)
console.print(encoder)


HuggingFaceEncoder(
    name='nlpaueb/legal-bert-base-uncased',
    score_threshold=0.5,
    type='huggingface',
    tokenizer_kwargs={},
    model_kwargs={},
    device='cpu'
)

In [7]:
from semantic_chunkers import StatisticalChunker
import logging

logging.disable(logging.CRITICAL)

chunker = StatisticalChunker(
    encoder=encoder,
    min_split_tokens=100,
    max_split_tokens=500,
)
console.print(chunker)

StatisticalChunker(
    name='statistical_chunker',
    encoder=HuggingFaceEncoder(
        name='nlpaueb/legal-bert-base-uncased',
        score_threshold=0.5,
        type='huggingface',
        tokenizer_kwargs={},
        model_kwargs={},
        device='cpu'
    ),
    threshold_adjustment=0.01,
    dynamic_threshold=True,
    window_size=5,
    plot_chunks=False,
    min_split_tokens=100,
    max_split_tokens=500,
    split_tokens_tolerance=10,
    enable_statistics=False
)

### Chunking the full document text

We will concatenate the text from all the pages of the document. We will insert the page number between the pages to allow the retrieval and then then the generation steps to create direct citation to the relevant page in the long document.

In [8]:
concatenated_text = " ".join([page["text"] + f"<page_break_{i}>" for i, page in enumerate(md_text)])

chunks = chunker(docs=[concatenated_text])


How many chunks were created?

In [9]:
len(chunks[0])

2336

Let's print a random chunk:

In [10]:
console.print(chunks[0][5])

Chunk(
    splits=[
        '“',
        'Porter explained, [I]t is often our professional legal opinion',
        'that a loan modification is in the best interests of our',
        'clients.',
        'In such cases, it would be unethical not to assist',
        'clients with these services.”',
        '- Porter states the “mailer received with the complaint',
        'is the only solicitation! ] used by PLC in WA.” Porter',
        'identified eight Washington residents PLC assisted with',
        'residential loan modifications and provided a “copy of a',
        '-----',
        '<page_break_41> solicitation to Washington consumers.” Porter admitted',
        'charging each of the eight Washington residents “$3997 [+]',
        'a monthly maintenance fee for loan modification services',
        'rendered.” Porter identified Christopher Jason Mercado as',
        'the Washington attorney who provided the residential loan',
        'modification services to Washington residents.',
        '- Porter claimed the attorney exemption to the Mort-',
        'gage Broker Practices Act (MBPA), chapter 19.146 RCW,',
        'applied and filed a “Claim of Non-Applicability of the',
        'Mortgage Broker Practices Act.” Porter asserted PLC was',
        'not subject to the MBPA because “Porter Law Center assists',
        'clients with application for loan modification as part of the',
        'licensed practice of law in the State of Washington.” DFI',
        'investigators attempted to contact the eight Washington',
        'residents Porter identified and “ultimately spoke to two of'
    ],
    is_triggered=True,
    triggered_score=0.7121353335887549,
    token_count=253,
    metadata=None
)

What is the average numebr of tokens in the chunks?

In [11]:
sum(chunk.token_count for chunk in chunks[0]) / len(chunks[0])

172.47217465753425

## Enrich the chunk with context and metadata

We will iterate over all the chunks. This can take some time based on the number of chunks.

Since we want to be able to process a large number documents in our RAG system, we need to create a UUID that will used as the ID of the chunk within the vector database. The UUID is comprised of the URL of the document and the chunk index. This structure allows you to get a specific chunk index directly, whick will be improtant in the augmentation phase.

In [12]:
import uuid
import re

doc_url = url
title = md_text[0]["metadata"]["title"]
# Enrich the metadata with filters that are relevant for future retrieval queries.
state = "Washington"

from tqdm import tqdm

def generate_uuid(doc_url, i):
    return str(uuid.uuid5(uuid.NAMESPACE_URL, f"{doc_url}/{i}"))

corpus_json = []
for i, chunk in tqdm(enumerate(chunks[0]), total=len(chunks[0]), desc="Processing chunks"):
    chunk_text = ' '.join(chunk.splits)
    page_match = re.search(r'<page_break_(\d+)>', chunk_text)
    page = page_match.group(1) if page_match else 0
    chunk_uuid = generate_uuid(doc_url, i)
    corpus_json.append({
        "id": chunk_uuid,
        "document": chunk_text,
        "embedding": encoder([f"{title} \n {chunk_text}"])[0],
        "metadata" : {
            "title": title,
            "state": state,
            "doc_url": doc_url,
            "chunk_index": i,
            "page": page,
        }
    })


Processing chunks:   0%|          | 0/2336 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing chunks: 100%|██████████| 2336/2336 [03:35<00:00, 10.83it/s]


In [ ]:
console.print(corpus_json[:2])

## Loading into a Vector Database

You will use a new vector data, [Chroma](https://github.com/chroma-core/chroma). It can illustrate the modularity of the RAG application, and the similar concepts across the providers.

### Creating the collection 

You will use the default values for this simpler exercise.

In [14]:
import chromadb
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

# Create collection. get_collection, create_collection, delete_collection also available!
collection_name = "legal-pdfs"
collection = client.get_or_create_collection(collection_name)


### Unserting the documents

We will use the embedding, metadata and documents that were calculated above.

In [15]:
collection.add(
    documents=[obj["document"] for obj in corpus_json],
    embeddings=[obj["embedding"] for obj in corpus_json],
    metadatas=[obj["metadata"] for obj in corpus_json],
    ids=[obj["id"] for obj in corpus_json]
)

### Query the vector collection

We will add an example filter to the query based on the metadata that we created for each chunk (`{"state": "Washington"}`).

In [16]:
query_text = "cases about loan default"
query_embedding = encoder([query_text])

In [17]:
hits = collection.query(
    query_embeddings=query_embedding,
    n_results=5,
    where={"state": "Washington"},
)


## Augmentation Step

We suspect that the chunk context is too small and we want to concatenate the chunks around it, before we send the text to the generation step. 

In [18]:
console.print(hits)

{
    'ids': [
        [
            'e6e5a091-a785-5ee1-9583-c1631a5ea1c8',
            '0771fe60-9611-5cb9-9043-c01bf9824ffa',
            'd8880a1b-84bf-5f0a-85ec-b3f4f818ed21',
            'ce09c735-b77e-5183-bcf6-7fdf72bb344d',
            '370e45fb-0625-5cb2-bcce-39adc25140a6'
        ]
    ],
    'distances': [
        [0.7120084762573242, 0.7148486375808716, 0.7168563008308411, 0.7176553606987, 0.7205198407173157]
    ],
    'metadatas': [
        [
            {
                'chunk_index': 1456,
                'doc_url': 'https://static.case.law/wash-app/196.pdf',
                'page': '684',
                'state': 'Washington',
                'title': 'Washington Appellate Reports volume 196'
            },
            {
                'chunk_index': 383,
                'doc_url': 'https://static.case.law/wash-app/196.pdf',
                'page': '205',
                'state': 'Washington',
                'title': 'Washington Appellate Reports volume 196'
            },
            {
                'chunk_index': 240,
                'doc_url': 'https://static.case.law/wash-app/196.pdf',
                'page': 0,
                'state': 'Washington',
                'title': 'Washington Appellate Reports volume 196'
            },
            {
                'chunk_index': 52,
                'doc_url': 'https://static.case.law/wash-app/196.pdf',
                'page': 0,
                'state': 'Washington',
                'title': 'Washington Appellate Reports volume 196'
            },
            {
                'chunk_index': 221,
                'doc_url': 'https://static.case.law/wash-app/196.pdf',
                'page': '129',
                'state': 'Washington',
                'title': 'Washington Appellate Reports volume 196'
            }
        ]
    ],
    'embeddings': None,
    'documents': [
        [
            '----- <page_break_684> raise the issue of appellate costs in briefing or a motion for reconsideration 
consistently with Sinclair.2 - 0 Our Supreme Court has acknowledged the difficult issues surrounding the assignment
of costs to indigent defendants. In State v. Blazina, the court catalogued the potential impediments to meaningful 
postincarceration re- integration caused by burdening indigent defendants with costs that will only swell with 
interest under a low payment plan. 182 Wn.2d 827, 344 P.3d 680 (2015).',
            'We should take the current while it serves. For these reasons, I dissent. Reconsideration granted 
October 31, 2016 to waive ap- pellate costs. Review denied at 188 Wn.2d 1008 (2017). 8 Perhaps the most obvious 
change made by Blazina was its modification of prior ripeness doctrine by allowing a defendant to challenge ability
to pay LFOs before enforcement. As the majority points out, though, Dove’s judgment and sentence stated that LFO 
payments would commence immediately. For this ----- <page_break_205> [No. 73268-4-I. Division One. October 3, 
2016.]',
            'Both Reilly and Ide, in dicta, imply that an exemption based on low income is reasonable and proper. -
1 I recognize that Town of Tekoa v. Reilly concerns a full exemption, not a reduction in property value for pur- 
poses of assessing taxes. Nevertheless, I find this distinc- tion unimportant in distinguishing Reilly. If a city 
may impart a full exemption to low income property owners, the city should enjoy the power to convey a partial 
exemption. A partial exemption is less harmful to other taxpayers. - 2 The city of Spokane relies on RCW 
35A.11.020,',
            'Once again, the parties2 agreed in the trial court that it was.3 That conclu- sion is unassailable. 
The policy of this state, expressed in the ethics in public service act, ch. 42.52 ROW,is that public resources are
to be used for official public business rather than for personal benefit. See Knudsen, 156 Wn. App. at 860-63 
(determining that e-mail sent to encourage others to lobby legislature violated de min

### Augmenting the search result

You will iterate over all the search results and prepare them to the generation step. The main augmentation is the concatenation of the sourounding chunks text.

In [19]:
# define a variable to hold the search results with specific fields
search_results = []

for document, metadata in zip(hits["documents"][0], hits["metadatas"][0]):
    doc_url = metadata["doc_url"]
    chunk_index = metadata["chunk_index"]
    doc_id = generate_uuid(doc_url, chunk_index)
    # Calculate the chunk IDs of the previous and next chunks
    previous_chunk_id = generate_uuid(doc_url, chunk_index-1)
    next_chunk_id = generate_uuid(doc_url, chunk_index+1)
    # Get the chunks from the vector collection with the chunk ids.
    previous_chunk = collection.get(ids=[previous_chunk_id])
    next_chunk = collection.get(ids=[next_chunk_id])
    search_results.append({
        # Concatenate the previous, current, and next document chunks to form a single document
        "document": f"{previous_chunk['documents'][0]} {document} {next_chunk['documents'][0]}",
        "metadata": metadata,
    })

Let's print the first search result, before sending it to the generation model:

In [20]:
console.print(search_results[0])

{
    'document': 'v. Stump, 185 Wn.2d 454, 460, 374 P.3d 89 (2016). “We determine that intent by examining the 
rule’s plain lan- guage not in isolation but in context, considering related provisions, and in light of the 
statutory or rule-making scheme as a whole.” Id. Furthermore, our Supreme Court has noted that “RAP 14.2 is not the
exclusive means for dealing with all situations resulting in an affirmance of a trial court decision,” and that 
“[t]he RAPs are instead designed to allocate appellate costs in a fair and equitable manner depending on the 
realities of the case.”Id. at 461. Finally, we may invoke RAP 1.2(c) to waive or alter the provisions of the Rules 
of Appellate Procedure in order to serve the ends of justice. - The strict interpretation of RAP 14.2 would require
a defendant to object to appellate costs either in briefing, before even knowing which party prevails, or on 
reconsideration, when the defendant may or may not know if the State is requesting costs. Only with the commission-
er’s decision is there an order imposing costs on a defendant who did not prevail on appeal.1 That, then, is a 
fair, logical, and appropriate time to require a defendant to challenge those costs, although a defendant may 
continue to properly 1 Both a motion for reconsideration and an objection to cost bill generally must ----- 
<page_break_684> raise the issue of appellate costs in briefing or a motion for reconsideration consistently with 
Sinclair.2 - 0 Our Supreme Court has acknowledged the difficult issues surrounding the assignment of costs to 
indigent defendants. In State v. Blazina, the court catalogued the potential impediments to meaningful 
postincarceration re- integration caused by burdening indigent defendants with costs that will only swell with 
interest under a low payment plan. 182 Wn.2d 827, 344 P.3d 680 (2015). The court explained that indigent offenders 
are disproportionately weighed down by legal costs compared to wealthier offend- ers, who are less likely to fall 
behind on payments and become subject to additional interest and fees. Id. at 836. Once encumbered by legal costs, 
an indigent offender is likely to encounter problems securing employment, hous- ing, or financing because 
background checks will indicate an active court record for unpaid costs. Id. at 837. These difficulties are similar
to, if not the same as, the difficulties implicated in Grant’s challenge. - 1 As already noted, a motion to modify 
is generally the first opportunity to challenge an award of appellate costs after the award has been made. That 
timing along with the problems caused by levying court costs on the indigent, the limited holding of Sinclair, and 
the legislative intent that the appellate court exercise discretion over imposing appel- late costs all counsel 
that RAP 14.2 should be read to allow Grant to challenge appellate costs for the first time in his motion to modify
the commissioner’s ruling where he has previously filed an objection to the cost bill. - 2 In addition, we have 
authority to determine “costs in all cases after the filing of a decision terminating review.” RAP 14.1(a) 
(emphasis added). To the extent that a com- missioner’s ruling on costs may be appealed to the appellate court 
under RAP 17.7, we thus retain ultimate discretion as 2A new proposed amendment to RAP 14.2 is anticipated from the
Supreme',
    'metadata': {
        'chunk_index': 1456,
        'doc_url': 'https://static.case.law/wash-app/196.pdf',
        'page': '684',
        'state': 'Washington',
        'title': 'Washington Appellate Reports volume 196'
    }
}

In [22]:
from openai import OpenAI
from rich.panel import Panel
from rich.text import Text

client = OpenAI()
system_message = """
You are a paralegal specialist. 
Your top priority is to help lawyers find information in a large legal corpus and help them with their requests. 
Please include citation to the document and the page number.
"""
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": query_text},
        {"role": "assistant", "content": str(search_results)}
    ]
)

response_text = Text(completion.choices[0].message.content)
styled_panel = Panel(
    response_text,
    title=f"Reply to '{query_text}'",
    expand=False,
    border_style="bright_yellow",
    padding=(1, 1)
)

console.print(styled_panel)

╭────────────────────────────────────── Reply to 'cases about loan default' ──────────────────────────────────────╮
│                                                                                                                 │
│ Here are some cases that discuss issues of loan default:                                                        │
│                                                                                                                 │
│ 1. v. Stump: The case examines a rule in a particular context, considering related provisions and the whole     │
│ statutory or rule-making scheme. (Washington Appellate Reports volume 196, Page 684)                            │
│ [Source](https://static.case.law/wash-app/196.pdf)                                                              │
│                                                                                                                 │
│ 2. Hassan Farah et al., v. Hertz Transporting, Inc., et al.: The details of this case aren't provided in the    │
│ given excerpt but it might be relevant for you to look up more details. (Washington Appellate Reports volume    │
│ 196, Page 205) [Source](https://static.case.law/wash-app/196.pdf)                                               │
│                                                                                                                 │
│ 3. Town of Tekoa v. Reilly: This case is discussing property tax exemptions, which may be pertinent if there's  │
│ a connection to property involved in the loan default. (Washington Appellate Reports volume 196, Page 136)      │
│ [Source](https://static.case.law/wash-app/196.pdf))                                                             │
│                                                                                                                 │
│ These cases may give some insight into handling similar situations involving loan defaults. Please provide more │
│ specific details if you need cases relating to a particular aspect of loan default.                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯